In [38]:
from langchain.document_loaders.csv_loader import CSVLoader

In [39]:
loader = CSVLoader(file_path="./prompts.csv")
data = loader.load_and_split()[:20]
data

[Document(page_content='act: Linux Terminal\nprompt: I want you to act as a linux terminal. I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. do not write explanations. do not type commands unless I instruct you to do so. when i need to tell you something in english, i will do so by putting text inside curly brackets {like this}. my first command is pwd', metadata={'source': './prompts.csv', 'row': 0}),
 Document(page_content='act: English Translator and Improver\nprompt: I want you to act as an English translator, spelling corrector and improver. I will speak to you in any language and you will detect the language, translate it and answer in the corrected and improved version of my text, in English. I want you to replace my simplified A0-level words and sentences with more beautiful and elegant, upper level English words and sentences. Keep the meaning same, but 

In [40]:
from IPython.display import Markdown

Markdown(data[0].page_content)

act: Linux Terminal
prompt: I want you to act as a linux terminal. I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. do not write explanations. do not type commands unless I instruct you to do so. when i need to tell you something in english, i will do so by putting text inside curly brackets {like this}. my first command is pwd

In [41]:
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = Ollama(
    model="llama2",
    verbose=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
) # allow for streaming the response

In [42]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100) # you should experiment with!
all_splits = text_splitter.split_documents(data)

In [43]:
# Embed and store
from langchain.vectorstores import Chroma
from langchain.embeddings import OllamaEmbeddings  # We can also try Ollama embeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OllamaEmbeddings(model="llama2"),persist_directory="lucas-old-test")

In [44]:
question = "Prompt about writing?"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [45]:
Markdown(docs[0].page_content)

act: Novelist
prompt: I want you to act as a novelist. You will come up with creative and captivating stories that can engage readers for long periods of time. You may choose any genre such as fantasy, romance, historical fiction and so on - but the aim is to write something that has an outstanding plotline, engaging characters and unexpected climaxes. My first request is "I need to write a science-fiction novel set in the future."

In [ ]:
"""youtube chapters section\nprompt: Given this video transcription: {transcription} you will write a Youtube chapter section with timeline stamps for the major parts of the video. The format should be: \\n\\n <double:digit timestamp>:<brief description of this section>'"""

In [46]:
# QA chain
from langchain.chains import RetrievalQA

# RAG prompt
from langchain import hub

In [47]:
QA_CHAIN_PROMPT = hub.pull("rlm/rag-prompt-llama")

print(QA_CHAIN_PROMPT)

input_variables=['context', 'question'] metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt-llama', 'lc_hub_commit_hash': '693a2db5447e3b58c060a6ac02758dc7f1aaaaa4ee6214d127bf70b443158630'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="[INST]<<SYS>> You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.<</SYS>> \nQuestion: {question} \nContext: {context} \nAnswer: [/INST]"))]


In [49]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

question = "prompt about writing"
result = qa_chain.invoke({"query": question})

Of course! I'd be happy to help you create an advertising campaign for a new energy drink targeting young adults aged 18-30. Here's a potential campaign strategy based on the context you provided:

Target Audience: Young adults aged 18-30 who lead active lifestyles and are looking for a boost of energy to keep them going throughout the day. This group is likely to be busy with work, school, or other activities, and may not have the time or energy to prepare healthy meals or snacks.

Key Messages: Our energy drink is specifically formulated to provide a quick and sustained energy boost without the crash that can come from other energy drinks. It's made with all-natural ingredients and contains no artificial flavors or sweeteners, making it a healthier choice for young adults who want to stay energized without sacrificing their well-being.

Media Channels: To reach our target audience, we will use a combination of online and offline marketing channels. These may include:

* Social media 

Cool alternative option using `CTransformers` for setting up the llama2 llm:

- https://levelup.gitconnected.com/talk-to-your-csv-llama2-how-to-use-llama2-and-langchain-69012c5ff653